<a href="https://colab.research.google.com/github/AbdelrahmanTamer11/Named-Entity-Recognition-NER-Corpus/blob/main/Named_Entity_Recognition_(NER)_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd

# Load the dataset
data_path = '/content/ner.csv'
data = pd.read_csv(data_path)

# No need for ast.literal_eval, directly use the 'Sentence' column and split the sentences into words
sentences = [sentence.split() for sentence in data['Sentence'].tolist()]
tags = [tag.strip('[]').replace("'", "").split(', ') for tag in data['Tag'].tolist()]



In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create a vocabulary for words and tags
words = list(set([word for sentence in sentences for word in sentence]))
n_words = len(words)

tags_set = list(set([tag for tag_list in tags for tag in tag_list]))
n_tags = len(tags_set)

word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags_set)}

# Convert sentences and labels into indices
X = [[word2idx[w] for w in s] for s in sentences]
y = [[tag2idx[t] for t in t_list] for t_list in tags]

# Padding the sequences
max_len = 50
X = pad_sequences(X, maxlen=max_len, padding="post")
y = pad_sequences(y, maxlen=max_len, padding="post")


In [33]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

# Define the maximum sequence length
max_len = 50

# Pad the input sentences and tag sequences
X = pad_sequences([[word2idx.get(w, 0) for w in s] for s in sentences], maxlen=max_len, padding="post")
y = pad_sequences([[tag2idx.get(t, 0) for t in tag_seq] for tag_seq in tags], maxlen=max_len, padding="post")

# Convert target labels (y) to categorical format
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])

# Build the NER model
model = Sequential([
    Embedding(input_dim=n_words, output_dim=50, input_length=max_len),
    Dropout(0.1),
    Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1)),
    TimeDistributed(Dense(n_tags, activation="softmax"))
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(X, y, batch_size=32, epochs=5, validation_split=0.2, verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 168s 134ms/step - accuracy: 0.9240 - loss: 0.3252 - val_accuracy: 0.9826 - val_loss: 0.0582
Epoch 2/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 198s 131ms/step - accuracy: 0.9853 - loss: 0.0498 - val_accuracy: 0.9853 - val_loss: 0.0476
Epoch 3/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 159s 132ms/step - accuracy: 0.9890 - loss: 0.0357 - val_accuracy: 0.9859 - val_loss: 0.0465
Epoch 4/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 157s 131ms/step - accuracy: 0.9906 - loss: 0.0297 - val_accuracy: 0.9861 - val_loss: 0.0467
Epoch 5/5
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 201s 130ms/step - accuracy: 0.9918 - loss: 0.0252 - val_accuracy: 0.9861 - val_loss: 0.0476


In [34]:
from seqeval.metrics import classification_report

# Predict and evaluate
y_pred = model.predict(X)

# Convert indices back to entity labels
idx2tag = {i: w for w, i in tag2idx.items()}

def get_tags(y_pred, y_true):
    pred_tags = [[idx2tag[np.argmax(pred)] for pred in p] for p in y_pred]
    true_tags = [[idx2tag[np.argmax(true)] for true in t] for t in y_true]
    return pred_tags, true_tags

# Get predictions
pred_tags, true_tags = get_tags(y_pred, y)

# Print the evaluation report
print(classification_report(true_tags, pred_tags))


1499/1499 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step
              precision    recall  f1-score   support

         art       0.49      0.34      0.40       401
         eve       0.44      0.44      0.44       307
         geo       0.89      0.93      0.91     37633
         gpe       0.97      0.95      0.96     15859
         nat       0.62      0.35      0.45       199
         org       0.83      0.78      0.80     20130
         per       0.82      0.86      0.84     16978
         tim       1.00      1.00      1.00   1370401

   micro avg       0.99      0.99      0.99   1461908
   macro avg       0.76      0.71      0.73   1461908
weighted avg       0.99      0.99      0.99   1461908



In [35]:
import numpy as np

# Function to get the predicted tags for a given sentence
def predict_entities(sentence, model, word2idx, idx2tag, max_len):
    # Tokenize and pad the sentence
    tokens = sentence.split()
    tokenized_sentence = [word2idx.get(w, 0) for w in tokens]  # Convert words to indices
    padded_sentence = pad_sequences([tokenized_sentence], maxlen=max_len, padding='post')

    # Get predictions
    pred = model.predict(padded_sentence)

    # Convert predictions back to tags
    pred_tags = [idx2tag[np.argmax(p)] for p in pred[0]]

    # Pair each word with its predicted tag
    result = list(zip(tokens, pred_tags[:len(tokens)]))  # Cut off padding
    return result

# Example usage
idx2tag = {i: t for t, i in tag2idx.items()}  # Reverse the tag dictionary

# Test the function with an example sentence
sentence = 'Google'
predicted_entities = predict_entities(sentence, model, word2idx, idx2tag, max_len)

# Print the result
for word, tag in predicted_entities:
    print(f"{word}: {tag}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Google: B-org
